# Tacotron: Towards End-to-End Speech Synthesis

- **Tacotron**: **Attention**을 포함하는 **seq2seq**를 기반으로 하는 **end-to-end** 생성 **TTS 모델**
    - <텍스트, 오디오> 쌍을 사용하여 완전히 처음부터 학습할 수 있다!

# Model Architecture

- **모델 전체 구조**
<img src="01.png" width=60% height=60%>

## 1. CBHG Module

먼저, **1-D Convolutional Bank**, **Highway Network**, **Bidirectional GRU**가 무엇인지 알아보자.

### 1.1 1-D Convolutional Bank

- **1-D Convolutional Bank**: 시퀀스에서 유용한 정보를 추출하기 위해 **여러 개의 1-D convolutional filters**를 적용한 블록
    - 1-D Convolutional Filters: 크기가 $1, 2, \dots, K$인 필터 $K$개를 이용한다.
    - Stacking: 각 필터의 연산 결과를 Concatenation한다.

### 1.2 Highway Network

- **Highway Network**: 딥러닝에서 기울기 소실 문제를 해결하기 위해 제안된 신경망 구조
    - Plain Feedforward Network를 구성하는 Layer가 다음과 같은 연산을 한다고 하자.
        $$\mathbf{y} = H(\mathbf{x}, \mathbf{W_{H}})$$
        ($\mathbf{y}$: output, $H$: non-linear activation function, $\mathbf{x}$: input, $\mathbf{W_{H}}$: weights) 
    - Highway Network는 두 개의 게이트 $T$와 $C$가 추가되어 다음과 같이 Layer가 구성된다.
        $$\mathbf{y} = H(\mathbf{x}, \mathbf{W_{H}})\cdot T(\mathbf{x}, \mathbf{W_{T}})+\mathbf{x}\cdot C(\mathbf{x}, \mathbf{W_{C}})$$
        ($T$: **transform gate**, $C$: **carry gate**)
    - 흔히 두 게이트에는 **sigmoid function**을 이용하며, $C=1-T$로 주어진다.
        $$\mathbf{y} = H(\mathbf{x}, \mathbf{W_{H}})\cdot T(\mathbf{x}, \mathbf{W_{T}})+\mathbf{x}\cdot (1-T(\mathbf{x}, \mathbf{W_{T}}))$$  
    
        - 위의 식의 특별한 경우를 생각해보면 다음과 같이 쓸 수 있다. (실제로는 $T$가 시그모이드 함수이므로 이런 경우는 발생하지 않는다.)
        <img src="02-1.png" width=50% height=50%>
        <img src="02-2.png" width=50% height=50%>
        
        - 이를 통해 Highway Network는 Residual Connection과 유사한 방식으로 기울기 소실을 방지한다는 것을 알 수 있다.
        - 다만, Gate를 통해 input의 정보를 얼마나 보존할지 결정하므로 정보의 유실을 막으면서 기울기 소실을 방지한다고 할 수 있다.

### 1.3 Bidirectional GRU

- **Bidirectional RNN**: 앞뒤 방향에서 각각의 입력을 처리하여, 앞뒤의 문맥 정보를 모두 활용할 수 있는 RNN
    <img src="03.png" width=70% height=70%>
    
    - 두 RNN의 출력은 Concat한다.

- **GRU**: LSTM과 마찬가지로 기울기 소실 문제를 해결하기 위해 고안된 RNN
    - LSTM vs GRU
    <img src="04.png" width=70% height=70%>
    <img src="05.png" width=70% height=70%>
    - GRU가 LSTM보다 **매개변수가 적어서** 학습 속도가 빠르고 작은 데이터셋에서도 좋은 성능을 보인다.

### 1.4 CHBG Module의 구조

- **CBHG Module**: 1-D **C**onvolution **B**ank + **H**ighway network + Bidirectional **G**RU로 구성되어 시퀀스로부터 표현을 추출해내는 강력한 모듈
<img src="06.png" width=60% height=60%>

    - CBHG Module은 Encoder와 Decoder에 모두 사용된다.
    - 다음 순서로 진행된다.
        1. **1-D Convolutional Bank**: 지역적인(local) 정보와 문맥(contextual) 정보를 추출해낸다.
        2. Max pooling: local invariance(지역 불변성, 문맥이 달라져도 변하지 않는 부분)를 증가시키는 역할을 한다. (stride=1)
        3. 1-D Convolutional Layers: 고정된 크기의 필터를 이용해서 벡터의 크기를 조절한다.
        4. Residual Connection: 기울기 소실 문제 해결, 더 빠른 수렴 속도
        5. **Highway Network**: 고수준(high-level)의 특징(feature)을 추출한다.
        6. **Bidirectional GRU**: 앞뒤로 순차적인 특징을 추출한다.
    - 이때 모든 convolutional layers에 **batch normalization**이 적용되었다.

## 2. Encoder

- **Encoder 전체 구조**
<img src="07.png" width=70% height=70%>

    - Encoder는 다음 순서로 진행된다.
        1. 텍스트를 문자 단위로 나눈 뒤 One-hot Encoding - Embedding한다.
        2. Bottleneck layer와 dropout으로 이루어진 **Pre-Net**을 통과한다.
            - **Pre-Net**: 각 층마다 Dropout이 있는 2층의 Fully Connected Layer로 구성된 신경망.
            - Pre-Net은 수렴을 도와주고 일반화를 향상시키는 역할을 한다.
        3. **CBHG Module**을 지나 최종적인 Encoder representation으로 변환한다.
            - 이때 CBHG Module은 다층 RNN보다 과적합을 줄이고 mispronounciations(잘못된 발음)을 줄인다.

## 3. Decoder

### 3.1 Mel-scale Spectrogram

- **Spectrogram(스펙트로그램)**: 오디오 신호의 진폭 정보를 시간-주파수 영역으로 시각화한 것
    - 기본적인 Spectrogram을 **Linear-scale Spectrogram**이라고도 한다.
    - Waveform(파형)에서는 오디오 신호의 시간, 진폭 정보를 확인할 수 있다.
    - Spectrogram(스펙트로그램)에서는 오디오 신호의 시간, 주파수, 진폭 정보를 모두 확인할 수 있다.
    <img src="08.png" width=70% height=70%>
    
    - 스펙트로그램은 FFT(Fast Fourier Transform)를 이용해서 구할 수 있다.

- **Mel-scale spectrogram**: 스펙트로그램에서 주파수 영역을 **Mel-scale**로 변환하여 표현한 것
    - **Mel-scale**: 사람이 인지하는 주파수 간격을 더 잘 반영하기 위해 일정한 간격으로 주파수 영역을 나눈 것
        - 사람은 고주파의 변화보다 <u>저주파의 변화를 잘 인지</u>하기 때문에, 저주파의 영역을 넓히는 방식을 이용한다. 그 식은 다음과 같다.
        $$m = 2595 \log_{10} \left(1 + \frac{f}{700} \right) $$
        ($m$: mel, $f$: hertz, 주파수 축을 mel 단위로 바꾼다.)
        - 역변환(Mel-spectrogram to Linear-spectrogram) 식은 다음과 같다.
        $$ f = 700\left(10^{\frac{m}{2595}} - 1\right)$$
    - Original Waveform vs Spectrogram vs Mel-spectrogram
        
<img src="09.png" width=100% height=100%>

### 3.2 Bahdanau Attention

- **Bahdanau Attention**: 이전의 Attention 구조를 개선한 방법 중 하나
    - 다음과 같은 구조를 갖는다.
    <img src="10.png" width=70% height=70%>
    
    - 식은 다음과 같다.
        1. Score: $\text{Score}(\mathbf{d}_{t}, \mathbf{h}) = \mathbf{W}^{T} \tanh (\mathbf{W_{d}}\mathbf{d}_{t}+\mathbf{W_{h}}\mathbf{h})$
        2. Attention Weight: $a_{t, j} = \frac{\exp(\text{Score}(\mathbf{d}_{t}, \mathbf{h}_{j}))}{\sum_{i=1}^{n} \exp(\text{Score}(\mathbf{d}_{t}, h_{i}))}$
        3. Context: $\mathbf{c}_{t} = \sum_{j=1}^{n} a_{t, j} h_{j}$  
        ($\mathbf{d}_{t}$: $t$ step에서의 Query 벡터, $\mathbf{h}$: Encoder의 output)

### 3.3 Decoder의 구조

- **Decoder 전체 구조**
<img src="11.png" width=70% height=70%>

    - **tanh attention(Bahdanau Attention) decoder**(seq2seq decoder 구조에 Attention을 추가한 것)을 기반으로 한다.
    - Decoder는 다음 순서로 진행된다.
        1. Decoder에는 처음에 **\<GO>**라는 all-zero frame이 입력된다.
        2. Pre-Net과 Attention RNN을 지나 **Query** 벡터 $\mathbf{d}$를 생성한다.
            - Attention RNN과 Decoder RNN에는 GRU가 사용되었다.
        3. Encoder의 hidden state(output) $\mathbf{h}$와 Query 벡터  $\mathbf{d}$를 **Attention**의 입력으로 받아서 **Context** 벡처 $\mathbf{c}$를 출력한다.
        4. Query 벡터 $\mathbf{d}$와 Context 벡터 $\mathbf{c}$를 concat하여 $(\mathbf{d}, \mathbf{c})$를 만든다.
        5. Decoder RNN과 Simple Fully-connected layer를 지나 **$r$개의 80-band mel-scale spectrogram**을 한번에 출력(예측)한다.
            - 이때 $r$을 **Redunction factor**라고 한다. (논문에서는 $r=2$로 설정했다.)
            - $r$개의 frames는 서로 겹치지 않는 연속된 frames이다!
            - 논문 저자들은 **$r$개의 frames를 출력하는 것**이 1) model size, training time, inference time을 줄일 수 있고, 2) alignment 학습에서 상당히 빠른 수렴 속도를 보인다는 것을 발견했다고 한다.
            - <font color="gray">$r$개의 frames를 출력하기 위해 FC-layer의 output을 $80\times r$로 하거나 FC-80-layer 뒤에 Conv1D를 추가하는 방식이 있는 것으로 추론된다.</font>
    - Decoder는 추론 과정에서의 input과 훈련 과정에서의 input이 서로 다르다.
        - 추론 과정에서의 Decoder input: \<GO>와 각 $t$ step에서의 $r$번째 예측(마지막 frame) 
        - 훈련 과정에서의 Decoder input: \<GO>와 **$r$의 배수**에 해당하는 **ground truth** frame (텍스트에 맞는 오디오의 80-band mel-scale spectrogram)

## 4. Post-Processing Net

- **Post-Processing Net 전체 구조**
<img src="12.png" width=100% height=100%>

    - **CBHG Module**의 역할: Mel-scale Spectrogram을 Linear-scale Spectrogram으로 변환하도록 학습한다.
        - 역변환 식을 이용하지 않고 CBHG Module을 이용한 이유는 seq2seq와 달리 **양방향 정보를 모두 활용**해서 각 예측 frame의 **오류를 보정**할 수 있기 때문이다.
    - **Griffin-Lim Algorithm**: Spectrogram에서 Waveform으로의 역변환을 수행하는 알고리즘 중 하나
        - Griffin-Lim Algorithm은 학습할 수 있는 가중치가 없고, 미분가능하다.
        - Griffin-Lim Algorithm을 사용하기 전에 Linear-scale Spectrogram을 **1.2 제곱**하면 신호의 잡음, 왜곡 등이 개선되었다고 한다.
        - 알고리즘을 약 50번 반복하면 Waveform으로 잘 수렴하는 것으로 관찰되었다.

[참고 논문]
1. Tacotron: https://arxiv.org/abs/1703.10135
2. Tacotron 한국어: https://scienceon.kisti.re.kr/srch/selectPORSrchArticle.do?cn=JAKO201811648108967
3. Highway Network: https://arxiv.org/abs/1505.00387

[참고 자료 / 사진 출처]
1. https://joungheekim.github.io/2020/09/25/paper-review/
2. https://hwrg.github.io/posts/Tacotron-%EB%94%A5%EB%9F%AC%EB%8B%9D-%EA%B8%B0%EB%B0%98%EC%9D%98-%EC%9D%8C%EC%84%B1%ED%95%A9%EC%84%B1-%EC%8B%9C%EB%8C%80%EB%A5%BC-%EC%95%8C%EB%A6%B0-TTS-%EB%AA%A8%EB%8D%B8/
3. https://khw11044.github.io/blog/papers/paper-etc/2021-02-01-tacotron1_expain/
4. Bidirectional RNN: https://ws-choi.github.io/blog-kor/bidirectional-rnn-in-pytorch/
5. GRU: https://sooftware.io/lstm_gru/
6. Spectrogram: https://ko.wikipedia.org/wiki/%EC%8A%A4%ED%8E%99%ED%8A%B8%EB%A1%9C%EA%B7%B8%EB%9E%A8
7. Bahdanau's Additive Style: https://www.linkedin.com/pulse/attention-mechanism-nlp-shailesh-jannu